# Database Table Loader
This notebook connects to the local PostgreSQL database, lists all tables, and loads a selected table into a pandas DataFrame.

In [1]:
import os
from dotenv import load_dotenv
import pandas as pd
from sqlalchemy import create_engine, inspect

# Load environment variables
load_dotenv(override=True)

DB_HOST = os.getenv('LOCAL_HOST')
DB_NAME = os.getenv('LOCAL_DB')
DB_USER = os.getenv('LOCAL_USER')
DB_PW = os.getenv('LOCAL_PW')
DB_PORT = os.getenv('LOCA_PORT', '5432')

engine = create_engine(f'postgresql://{DB_USER}:{DB_PW}@{DB_HOST}:{DB_PORT}/{DB_NAME}')

In [2]:
# List all tables in the database
inspector = inspect(engine)
tables = inspector.get_table_names()
print('Tables in database:', tables)

Tables in database: ['urban_data_expanded', 'urban_data_directory', 'census_data', 'location_data', 'urban_institute_data']


In [25]:
# Load all tables into pandas DataFrames
dfs = {}
for table in tables:
    df = pd.read_sql_table(table, engine)
    dfs[table] = df
    print(f"Loaded table: {table}, shape: {df.shape}")
if not tables:
    print('No tables found in the database.')

Loaded table: urban_data_expanded, shape: (220000, 33)
Loaded table: urban_data_directory, shape: (90000, 57)
Loaded table: census_data, shape: (333160, 16)
Loaded table: location_data, shape: (27796, 6)
Loaded table: urban_institute_data, shape: (310000, 8)


In [36]:
census_data = dfs['census_data']

school_directory = dfs['urban_data_directory']
school_directory = school_directory.query('`latitude`!="" and `longitude`!=""')
school_directory[['latitude', 'longitude']] = school_directory[['latitude', 'longitude']].astype(float)
school_data = dfs['urban_data_expanded']
location_data = dfs['location_data']


C:\Users\Leo\AppData\Local\Temp\ipykernel_35364\2160154767.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  school_directory[['latitude', 'longitude']] = school_directory[['latitude', 'longitude']].astype(float)


In [45]:
school_location = pd.merge(school_directory, 
         location_data, 
         on=['latitude', 'longitude'], 
         how='left'
)
school_facts = school_location[['city_mailing', 'county', 'state', 'zip_mailing', 
                 'leaid', "ncessch",'school_name','school_status','school_type',
                 'enrollment','year']]

In [49]:
school_assessments = school_data[['lea_name','leaid','ncessch','year', 
             'econ_disadvantaged', 'enrollment','grade',
             'math_test_num_valid', 'math_test_pct_prof_high', 'math_test_pct_prof_low', 'math_test_pct_prof_midpt',
             'migrant','race', 'read_test_num_valid', 'read_test_pct_prof_high', 'read_test_pct_prof_low', 'read_test_pct_prof_midpt',
             'sex']]

In [ ]:
pd.merge(school_facts, school_assessments, on=['leaid','ncessch','year'], how='right')

,city_mailing,county,state,zip_mailing,leaid,ncessch,school_name,school_status,school_type,enrollment_x,...,math_test_pct_prof_high,math_test_pct_prof_low,math_test_pct_prof_midpt,migrant,race,read_test_num_valid,read_test_pct_prof_high,read_test_pct_prof_low,read_test_pct_prof_midpt,sex
0,Birmingham,Jefferson County,Alabama,35220,0100002,010000200277,Sequoyah Sch - Chalkville Campus,1,4,-1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Clayton,Barbour County,Alabama,36016,0100002,010000201402,Eufaula Sch - Eufaula Campus,2,4,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Mt Meigs,Autauga County,Alabama,36057,0100002,010000201667,Camps,1,4,-1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Mt Meigs,Clarke County,Alabama,36057,0100002,010000201670,Det Ctr,1,4,-1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Mount Meigs,Montgomery County,Alabama,36057,0100002,010000201705,Wallace Sch - Mt Meigs Campus,1,4,-1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101159,Placerville,El Dorado County,California,95667,0615450,061545001963,Gold Trail,1,1,268,...,,,,,99,,,,,99
101160,Placerville,El Dorado County,California,95667,0615450,061545010200,Sutters Mill,1,1,230,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101161,Oroville,Butte County,California,95965,0615480,061548001964,Concow Elementary,1,1,72,...,,,,,99,,,,,99
101162,Goleta,Santa Barbara County,California,93117,0615510,061551001969,Ellwood Elementary,1,1,327,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
